In [1]:
import pandas as pd
import numpy as np
import requests
from lxml import html
import xml.etree.ElementTree as et
from decimal import Decimal
from ratelimit import limits, sleep_and_retry
import matplotlib.pyplot as plt

# Part I. - Scraping Rohlik website #

In [2]:
# ------------------ GET PRODUCT URLs
# define url - site map of all the products
url = 'https://www.rohlik.cz/sitemap.xml'

# create http response object
resp = requests.get(url)

# parse xml
tree = et.fromstring(resp.content)
child = tree.getchildren()

# extract product URLs
products = [i[0].text for i in child]

# convert to panda series (it supports regex)
products = pd.Series(products)

# use regex to extract URLs that contain products with their prices - defined as 7 numbers and text, divided by '-'
products = products[products.str.contains('https://www.rohlik.cz/[0-9]{1,8}-*')]

C:\Users\petra\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  # Remove the CWD from sys.path while we load stuff.


In [3]:
products[:10]

0    https://www.rohlik.cz/1296727-nivea-men-silver...
1    https://www.rohlik.cz/1296729-nivea-for-men-in...
2    https://www.rohlik.cz/1296749-nivea-intimo-sen...
3    https://www.rohlik.cz/1296751-nivea-creme-care...
4    https://www.rohlik.cz/1296953-odol-stoma-parad...
5    https://www.rohlik.cz/1296965-racio-chlebicky-...
6    https://www.rohlik.cz/1296977-nescafe-dolce-gu...
7    https://www.rohlik.cz/1297003-alpro-kokosovy-n...
8    https://www.rohlik.cz/1297005-alpro-sojovy-nap...
9    https://www.rohlik.cz/1297007-wasa-delikatess-...
dtype: object

In [4]:
# ------------------ TRY THE SCRIPT FOR SCRAPING PRICES ON 1 PRODUCT:
prod_1 = products[160]
pageContent = requests.get(prod_1).content
tree = html.fromstring(pageContent)

prod_name = tree.xpath('//*[@class="redirect_link disabled"]//text()')

nopromo = tree.xpath('//*[@class="currentPrice"]//text()')

if(len(nopromo) == 0):
  nopromo = tree.xpath('//*//del/text()')

if(len(nopromo) == 2 and ('cca' in nopromo[0])):
  nopromo = nopromo[1]

if((isinstance(nopromo, (list,)) and len(nopromo) > 1) or (len(nopromo) == 0)):
    nopromo = np.nan

promo = tree.xpath('//*[@class="actionPrice"]//text()')

if(len(promo) == 0):
    promo = np.nan 

prod_quantity = tree.xpath('//*[@class="detailQuantity"]//text()')
if(len(prod_quantity) == 0):
    prod_quantity = np.nan 

prod_category = tree.xpath('//*[contains(@class, "redirect_link") and contains(@href, "/c")]//text()')
prod_cat = '-'.join(prod_category)

# create dictionary from scraped data: 
data = {'Product name': prod_name,
'Regular price': nopromo,
'Promo price': promo, 
'Quantity': prod_quantity, 
'Category': prod_cat}

# if df does not exist, create it: 
df = pd.DataFrame(data)
# else append the data to the existing data frame:
#df = df.append(pd.DataFrame(data))

In [5]:
df

,Product name,Regular price,Promo price,Quantity,Category
0,HiPP Bio Boloňské lasagne,"69,90 Kč","59,90 Kč",250 g,Příkrmy-Masové


In [ ]:
# ----------- FOR LOOP 
df = pd.DataFrame()

for x in products[50:70]:
  pageContent = requests.get(x).content
  tree = html.fromstring(pageContent)
  prod_name = tree.xpath('//*[@class="redirect_link disabled"]//text()')

  nopromo = tree.xpath('//*[@class="currentPrice"]//text()')

  if(len(nopromo) == 0):
    nopromo = tree.xpath('//*[@class="actionPrice"]/del//text()')

  if(len(nopromo) == 2 and ('cca' in nopromo[0])):
    nopromo = nopromo[1]

  promo = tree.xpath('//*[@class="actionPrice"]//text()')

  if(len(promo) == 0):
    promo = np.nan 
  
  if(len(nopromo) == 0):
    nopromo = np.nan

  prod_quantity = tree.xpath('//*[@class="detailQuantity"]//text()')
  if(len(prod_quantity) == 0):
    prod_quantity = np.nan

  prod_category = tree.xpath('//*[@class="sc-1ywzolw-1 jQxyMp"]//text()')
  prod_cat = '-'.join(prod_category)

  data = {'Product name': prod_name,
          'Regular price': nopromo, 
          'Promo price': promo,
          'Quantity': prod_quantity, 
          'Category': prod_cat}
  df = df.append(pd.DataFrame(data))

In [15]:
# ----------- DEFINE A FUNCTION TO EXTRACT DATA
@limits(calls = 10, period = 60) # set limit on scraping: 10 products per minute ~ 25.5 hours of scraping
@sleep_and_retry 
def extract_rohlik(x):
    pageContent = requests.get(x).content
    tree = html.fromstring(pageContent)
    prod_name = tree.xpath('//*[@class="redirect_link disabled"]//text()')

    nopromo = tree.xpath('//*[@class="currentPrice"]//text()')

    if(len(nopromo) == 0):
      nopromo = tree.xpath('//*//del/text()')

    if(len(nopromo) == 2 and ('cca' in nopromo[0])):
      nopromo = nopromo[1]
    
    if((isinstance(nopromo, (list,)) and len(nopromo) > 1) or (len(nopromo) == 0)):
      nopromo = np.nan

    promo = tree.xpath('//*[@class="actionPrice"]//text()')

    if(len(promo) == 0):
      promo = np.nan 

    prod_quantity = tree.xpath('//*[@class="detailQuantity"]//text()')
    if(len(prod_quantity) == 0):
      prod_quantity = np.nan 

    prod_category = tree.xpath('//*[contains(@class, "redirect_link") and contains(@href, "/c")]//text()')
    prod_cat = '-'.join(prod_category)

    data = {'Product name': prod_name,
            'Regular price': nopromo, 
            'Promo price': promo,
            'Quantity': prod_quantity, 
            'Category': prod_cat, 
            'URL': x}
    return(pd.DataFrame(data))

In [17]:
# try the function:
a = [extract_rohlik(x) for x in products[160:170]]
pd.concat(a, axis = 0)

,Product name,Regular price,Promo price,Quantity,Category,URL
0,HiPP Bio Boloňské lasagne,"69,90 Kč","59,90 Kč",250 g,Příkrmy-Masové,https://www.rohlik.cz/1298009-hipp-bio-bolonsk...
0,HiPP Bio Jablečno - hroznová šťáva,"34,90 Kč",NaN,200 ml,Nápoje-Ovocné šťávy,https://www.rohlik.cz/1298011-hipp-bio-jablecn...
0,HiPP Bio Hrušková šťáva,"34,90 Kč",NaN,200 ml,Nápoje-Ovocné šťávy,https://www.rohlik.cz/1298013-hipp-bio-hruskov...
0,HiPP BIO Meruňkovo-Hruškový nektar,"34,90 Kč",NaN,200 ml,Nápoje-Ovocné šťávy,https://www.rohlik.cz/1298015-hipp-bio-merunko...
0,HiPP Bio Švestkový nektar,"34,90 Kč",NaN,200 ml,Nápoje-Ovocné šťávy,https://www.rohlik.cz/1298017-hipp-bio-svestko...
0,HiPP Bio Šťáva z červených plodů ovoce,"74,90 Kč",NaN,500 ml,Nápoje-Ovocné šťávy,https://www.rohlik.cz/1298019-hipp-bio-stava-z...
0,HiPP Bio Jablečno - hroznová šťáva,"74,90 Kč",NaN,500 ml,Nápoje-Ovocné šťávy,https://www.rohlik.cz/1298021-hipp-bio-jablecn...
0,HiPP Bio Jablečná šťáva s fenyklovým čajem,"49,90 Kč","46,90 Kč",500 ml,Nápoje-Ovocné šťávy,https://www.rohlik.cz/1298023-hipp-bio-jablecn...
0,HiPP Mama Nápoj v prášku pro kojící matky,"139,90 Kč",NaN,200 g,Pro maminky a těhotné-Kojící čaje a nápoje,https://www.rohlik.cz/1298025-hipp-mama-napoj-...
0,HiPP Mamasanft Masážní olej na strie,"249,90 Kč",NaN,100 ml,"Pro maminky a těhotné-Krémy na strie, celulitidu",https://www.rohlik.cz/1298029-hipp-mamasanft-m...


In [ ]:
# Data storing with try except
all_product_data = pd.DataFrame([])

for x in products[:100]:
    try:
        d = extract_rohlik(x) 
        all_product_data = all_product_data.append(d)
        
    except Exception as e:
        # If there is an error, store the error message and corresponding url to the dataframe
        error_frame = pd.DataFrame([[str(e), x]], columns = ['Error', 'URL'])
        all_product_data = all_product_data.append(error_frame)

all_product_data

In [ ]:
# Export data: 
all_product_data.to_csv('data_week_4a.csv', sep = ';', encoding='utf-8')

In [2]:
# Try to read the data (see if encoding works properly): 
try_data = pd.read_csv('data_week_4a.csv', sep = ';')
try_data.head()

FileNotFoundError: File b'data_week_4a.csv' does not exist

# Part II. - Loading and preparing data for analysis #

In [10]:
### Data load ###

# Set the number of weeks in which data were scraped
n_of_weeks = 3  

# Initialize empty dataframe with time column
data = pd.DataFrame(columns = ['Time'])

# Load parts a,b for every week, add time, append to the final dataframe
for x in range(1, n_of_weeks + 1):
    
    a = pd.read_csv('data_week_' + str(x) + 'a.csv', sep = ';', encoding = 'utf8')
    b = pd.read_csv('data_week_' + str(x) + 'b.csv', sep = ';', encoding = 'utf8')
    a['Time'] = x
    b['Time'] = x
    
    data = data.append(a, ignore_index = True, sort = True).append(b, ignore_index = True, sort = True)

In [11]:
# Check
data.head(3)

,Category,Error,Product name,Promo price,Quantity,Regular price,Time,URL,Unnamed: 0
0,Pánské-Kuličkové,NaN,Nivea Men Silver Protect Kuličkový antiperspirant,"84,90 Kč",50 ml,"89,90 Kč",1,https://www.rohlik.cz/1296727-nivea-men-silver...,0.0
1,Pánské-Ve spreji,NaN,Nivea For Men Invisible for black & white anti...,"84,90 Kč",150 ml,"89,90 Kč",1,https://www.rohlik.cz/1296729-nivea-for-men-in...,0.0
2,Dámské hygienické potřeby-Intimní hygiena,NaN,Nivea Intimo Sensitive sprchová emulze pro int...,NaN,250 ml,"119,90 Kč",1,https://www.rohlik.cz/1296749-nivea-intimo-sen...,0.0


In [12]:
# Dropping unnecessary columns
data = data[['Category', 'Product name', 'Regular price', 'Promo price', 'Quantity', 'Time']]

# Splitting category column into main category and subcategory
data[['Category', 'Subcategory']] = data.Category.str.split("-", n = 1, expand = True)

In [13]:
# Check
data.head(3)

,Category,Product name,Regular price,Promo price,Quantity,Time,Subcategory
0,Pánské,Nivea Men Silver Protect Kuličkový antiperspirant,"89,90 Kč","84,90 Kč",50 ml,1,Kuličkové
1,Pánské,Nivea For Men Invisible for black & white anti...,"89,90 Kč","84,90 Kč",150 ml,1,Ve spreji
2,Dámské hygienické potřeby,Nivea Intimo Sensitive sprchová emulze pro int...,"119,90 Kč",NaN,250 ml,1,Intimní hygiena


In [15]:
# Converting prices to numbers

# a) Trim "Kč" 
data['Promo price'] = data['Promo price'].str.replace('Kč', '')
data['Regular price'] = data['Regular price'].str.replace('Kč', '')

# b) Decimal point instead of comma
data['Promo price'] = data['Promo price'].str.replace(',', '.')
data['Regular price'] = data['Regular price'].str.replace(',', '.')

# c) Remove thousand separator in numbers
data['Promo price'].replace('\s+', '', regex = True, inplace = True)
data['Regular price'].replace('\s+', '', regex = True, inplace = True)

# d) Remove all non-decimal characters
data['Promo price'].replace(r'[^\d.]+', '', regex = True, inplace = True)
data['Regular price'].replace(r'[^\d.]+', '', regex = True, inplace = True)

# e) Convert to float type
data['Promo price'] = data['Promo price'].astype('float')
data['Regular price'] = data['Regular price'].astype('float')

# f) Round to 1 decimal place
data = data.round(1)

In [16]:
# Check
print(data.dtypes)
data.head(10)

Category          object
Product name      object
Regular price    float64
Promo price      float64
Quantity          object
Time              object
Subcategory       object
dtype: object


,Category,Product name,Regular price,Promo price,Quantity,Time,Subcategory
0,Pánské,Nivea Men Silver Protect Kuličkový antiperspirant,89.9,84.9,50 ml,1,Kuličkové
1,Pánské,Nivea For Men Invisible for black & white anti...,89.9,84.9,150 ml,1,Ve spreji
2,Dámské hygienické potřeby,Nivea Intimo Sensitive sprchová emulze pro int...,119.9,NaN,250 ml,1,Intimní hygiena
3,Mýdla,Nivea Creme Care tekuté mýdlo na ruce,59.9,NaN,250 ml,1,Tekutá
4,Ústní hygiena,Odol Stoma Paradentol Ústní voda pro zdravé dásně,89.9,84.9,500 ml,1,Ústní vody
5,Racio a Knäckebrot,RACIO Chlebíčky rýžové,13.9,NaN,130 g,1,Pufované pečivo
6,Káva,Nescafé Dolce Gusto Lungo Intenso 16ks,149.9,NaN,NaN,1,Kapsle a pody
7,Mléko a mléčné nápoje,Alpro Kokosový nápoj Original s rýží,69.9,NaN,1 l,1,Rostlinné nápoje
8,Mléko a mléčné nápoje,Alpro Sójový Nápoj Original,59.9,NaN,1 l,1,Rostlinné nápoje
9,Racio a Knäckebrot,Wasa Delikatess celozrnný žitný křupavý chléb,54.9,NaN,270 g,1,Knäckebrot


In [17]:
# Compute promo in % (NaN is produced if there was no promo price)
data['Discount in %'] = (((data['Regular price'] - data['Promo price']) / data['Regular price']) * 100).round(1)

## 0. GENERAL OVERVIEW ##

**How many categories, subcategories, and products can we find on Rohlik.cz website?**

In [18]:
data.nunique()[['Category', 'Subcategory', 'Product name']]

Category          219
Subcategory       811
Product name    14699
dtype: int64

On Rohlik.cz, we can find 211 product categories, which are further divided into 789 subcategories. In total, we can find 14 399 unique products on the Rohlik.cz website.

**How many discounts did Rohlik.cz offer during the analyzed time period?**

In [19]:
# Count promos during analyzed time period
data.count()[['Discount in %']]

Discount in %    6324
dtype: int64

During the analyzed time period, we could find 4 404 discount offers.

**Analysis structure**

Our analysis of Rohlik.cz data has the following structure:

**1. Category analysis**

&nbsp;&nbsp;  **A)** The cheapest and the most expensive product categories <br>
&nbsp;&nbsp;  **B)** Categories with the best product choice <br>
&nbsp;&nbsp;  **C)** Categories with the highest number of discounted products in the analyzed time period & categories with the best deals (highest average discount in %)<br>

**2. Product analysis**

&nbsp;&nbsp;  **A)** The most expensive and the cheapest products on Rohlik.cz   
&nbsp;&nbsp;  **B)** Products that were offered in the biggest discount (in %) during the analyzed time period<br>

**3. Pricing analysis**

&nbsp;&nbsp;  **A)** Pricing overview for all products in the whole analyzed time period
- Distribution of regular product prices
- Distribution of discounts (in %)<br>

&nbsp;&nbsp;  **B)** Pricing overview for selected products, development of prices over time
- Product selection
- Analysis of development of regular prices in time
- Analysis of development of promo prices in time
 
